Aprendizaje Automático - Práctica 1
Autores: RICARDO ANTONIO PAZOS VALERO - 100472303 / VICENTE ANTONIO BARBATO - 10043114

# 6. Problema de clasificación

### Primera Parte: Usar modelo final para comprobar predicciones de valores altos y valores bajos

In [5]:
# Importamos
from joblib import load
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Cargamos datos
data = pd.read_csv('data/wind_ava.csv')

# Cargamos el modelo final
final_model = load('modelo_final.pkl')

# Filtramos los datos para ver solo los datos metereológicos obtenidos para Sotavento 
data_filter = [columna for columna in data.columns if columna.endswith('.13') or columna == "energy"]
data_sotavento = data[data_filter]
data_sotavento = data_sotavento.copy()  # Hacemos una copia de los datospara evitar el SettingWithCopyWarning

# Definimos el tercer cuantil
energy_tq = data_sotavento['energy'].quantile(0.75)

# Creamos una nueva variable target - la definimos como una variable binaria (más fácil de trabajar con variables binarias)
# 1 - Clase alta
# 2 - Clase baja
data_sotavento['energy_level'] = (data_sotavento['energy'] > energy_tq).astype(int)

# Hacemos las predicciones con el modelo final
predictions = final_model.predict(data_sotavento.drop(columns=['energy', 'energy_level']))

# Separamos los datos en dos subsets de energía alta o energía baja
high_energy = data_sotavento[data_sotavento['energy_level'] == 1]
low_energy = data_sotavento[data_sotavento['energy_level'] == 0]

# Hacemos predicciones para cada subset por separado
high_energy_predictions = predictions[high_energy.index]
low_energy_predictions = predictions[low_energy.index]

# Evaluamos el modelo en cada subset por separado
high_energy_mae = mean_absolute_error(high_energy['energy'], high_energy_predictions)
high_energy_mse = mean_squared_error(high_energy['energy'], high_energy_predictions)
high_energy_r2 = r2_score(high_energy['energy'], high_energy_predictions)

low_energy_mae = mean_absolute_error(low_energy['energy'], low_energy_predictions)
low_energy_mse = mean_squared_error(low_energy['energy'], low_energy_predictions)
low_energy_r2 = r2_score(low_energy['energy'], low_energy_predictions)

# Comparamos métricas
print("Métricas - Energía alta:")
print("MAE:", high_energy_mae)
print("MSE:", high_energy_mse)
print("R^2:", high_energy_r2)

print("\nMétricas - Energía baja:")
print("MAE:", low_energy_mae)
print("MSE:", low_energy_mse)
print("R^2:", low_energy_r2)

Métricas - Energía alta:
MAE: 683.7764569877376
MSE: 632524.6271455635
R^2: -2.956877450284961

Métricas - Energía baja:
MAE: 345.8537427064807
MSE: 185738.59314795886
R^2: -0.8795894043467756


Observamos que tenemos mejores resultados en las predicciones de los valores de energía bajos

### Segunda Parte: Cambiar a un modelo de clasificación